In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
from statsmodels.tsa.statespace.tools import diff
from statsmodels.tsa.tsatools import add_lag
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OrdinalEncoder

In [2]:
data = pd.read_stata("../data/kotz/data/T_econ.dta")
data["T5_mean_diff"] = diff(data["T5_mean"])
data["T5_mean_diff_lag"] = np.insert(add_lag(data["T5_mean_diff"])[:,1], 0, np.NaN)
data["T5_seas_diff_mXT5_varm"] = data["T5_seas_diff_m"] * data["T5_varm"]
data["T5_mean_mXT5_mean_diff"] = data["T5_mean_m"] * data["T5_mean_diff"]
data["T5_mean_mXT5_mean_diff_lag"] = data["T5_mean_m"] * data["T5_mean_diff_lag"]

In [23]:
# make withheld data
data = data.dropna(axis=0).reset_index(drop=True)
data["stratify_key"] = str(data.ID) + "_" + str(data.yearn)
splits = StratifiedShuffleSplit(n_splits=1, test_size=.1).split(data, data.stratify_key)
for (train_split, test_split) in splits:
    training_rows = data.iloc[train_split]
    test_rows = data.iloc[test_split]
    training_rows.sort_values(by=["country","ID","yearn"]).reset_index(drop=True).to_csv("../data/kotz/data/kotz_data_train.csv")
    test_rows.sort_values(by=["country","ID","yearn"]).reset_index(drop=True).to_csv("../data/kotz/data/kotz_data_test.csv")

In [3]:
model = pd.read_pickle("../models/nature_reproduction/kotz_reproduction_covar_coefs_only.pkl")
t5_coef = model["t5_varm"].data.flatten() * np.std(data.dlgdp_pc_usd) / np.std(data.T5_varm) * 100
t5_seas_diff_mXt5_varm_data = model["t5_seas_diff_mXt5_varm"].data.flatten() * np.std(data.dlgdp_pc_usd) / np.std(data.T5_seas_diff_mXT5_varm) * 100

In [4]:
maskc=gpd.read_file('../data/kotz/data/gadm36_levels.gpkg')

In [5]:
mask=gpd.read_file('../data/kotz/data/gadm36_levels.gpkg',layer=1)

In [13]:
GID0list=pd.unique(mask.GID_0)
isolist=pd.unique(data.iso)

#empty list to hold values
proj=[]

#calculate marginal effects to plot 
for i in range(len(GID0list)):
	iso=GID0list[i]
	no_regions=len(mask.loc[mask.GID_0==iso])
	if iso in isolist:
		wlrd1list=pd.unique(data.loc[data.iso==iso,'wrld1id_1'])
		for j in range(no_regions):
			reg_no=j+1
			if reg_no in wlrd1list:	
				T_diff=data.loc[(data.iso==iso) & (data.wrld1id_1==reg_no),'T5_seas_diff_m'].mean()
				proj.append(sorted(t5_coef + t5_seas_diff_mXt5_varm_data*T_diff))
			else:	
				proj.append([np.nan]*len(t5_coef))
	else:
		for j in range(no_regions):
			proj.append([np.nan]*len(t5_coef))

# for quantile in range(5, 105, 10):
# for quantile in [5, 50, 95]:
# print("Quantile:", quantile)
# proj_quant_data = list(map(lambda x: np.quantile(x, quantile/100), proj))
quantile = "diff_95-5"
proj_quant_data = list(map(lambda x: (np.quantile(x, .95) - np.quantile(x, .05)), proj))

mask[f'T5_varm_marg_losses_quant_{quantile}']=proj_quant_data

#generate divergent colormap
colors1 = np.flip(plt.cm.Blues_r(np.linspace(0., 1, 100)),0)
colors3 = np.flip(plt.cm.Reds(np.linspace(0, 1, 100)),0)
colors2 = np.array([1,1,1,1])

vmax=5
vmin=-5
colors = np.vstack((colors3, colors2, colors1))

my_cmap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)
#my_cmap = shiftedColorMap(my_cmap, start=vmin, stop = vmax, midpoint=0, name='my_cmap')
divnorm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)

degree_sign= u'\N{DEGREE SIGN}'
C=degree_sign + 'C'

#simplify shapefile geometry for plotting
mask['geometry_simpl']=mask.geometry.simplify(tolerance=720/43200,preserve_topology=True)
mask_simp=mask.copy()
mask_simp['geometry']=mask_simp.geometry.simplify(tolerance=720/43200,preserve_topology=True)

degree_sign= u'\N{DEGREE SIGN}'
C=degree_sign + 'C'
fs=7
cm=1/2.54
vmin=0
vmax=5
i=5
plt.close()
fig, ax = plt.subplots(1, 1,figsize=(18*cm,(9+0.5)*cm))
#maskc.plot(ax=ax,edgecolor='black',linewidth=0.1,color='grey')
# mask_simp[~mask_simp[f'T5_varm_marg_losses_quant_{quantile}'].isnull()].plot(ax=ax,column=f'T5_varm_marg_losses_quant_{quantile}',cmap='Reds_r',vmin=vmin,vmax=vmax)
mask_simp[~mask_simp[f'T5_varm_marg_losses_quant_{quantile}'].isnull()].plot(ax=ax,column=f'T5_varm_marg_losses_quant_{quantile}',cmap='Reds',vmin=vmin,vmax=vmax)
mask_simp[mask_simp[f'T5_varm_marg_losses_quant_{quantile}'].isnull()].plot(ax=ax,color='grey')
ax.set_ylim([-60,85])
ax.set_xlim([-180,180])
ax.tick_params(labelsize='large')
# sm = plt.cm.ScalarMappable(cmap='Reds_r', norm=plt.Normalize(vmin=vmin,vmax=vmax))
sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=vmin,vmax=vmax))
sm._A = []
ax.tick_params(bottom=False,left=False,labelbottom=False,labelleft=False)
cbar = fig.colorbar(sm,ax = ax,orientation='horizontal',fraction=0.046, pad=0.05)
cbar.set_label('Change in growth rates per extra degree of day-to-day temperature variability (%-points)',rotation=0, labelpad = 12, fontsize=fs)
cbar.ax.tick_params(labelsize=fs)
plt.savefig(f'../figures/kotz_maps/kotz_fig2_quant_{quantile}.pdf',dpi=600,bbox_inches='tight',pad_inches=0)
plt.close()